# RSA

Rivest Shamir Adleman (RSA) jest jednym z pierwszych algorytmów asymetrycznych. Został zaprojektowany w 1977 roku i jest używany do dzisiejszego dnia. Zyskał akceptację zarówno NISTu jak i organizacji ISO/IEC oraz RFC. RSA posiada parę kluczy - publiczny oraz prywatny. Publiczny klucz może być znany każdemu i służy on do operacji szyfrowania. Klucz prywatny jest znany tylko i wyłącznie instancji, która klucze generowała. Ta sama instancja jako jedna jedyna ma możliwość odszyfrowania kryptogramów.

RSA umożliwia także tworzenie podpisów cyfrowych (z ang *Digital Signatures*, czyli *DS*). Podpis cyfrowy to dodatkowy blok informacji dołączony do wiadomości, który zapewnia:
1. *Integrity* - integralność wiadomości, czyli potwierdzenie, że nie była ona w żaden sposób modyfikowana.
2. *Authentication* - autentykacje podpisującego, czyli potwierdzenie jego tożsamości.
3. *Non-repudiation* - czyli wysyłający podpisaną wiadomość nie ma możliwości zaprzeczenia faktu, że to on ją podpisał, natomiast otrzymujący wiadomość nie ma możliwości zaprzeczenia faktu, iż to on ją zweryfikował. 

Samo haszowanie wiadomości zapewnia tylko *integirty*, natomiast utworzenie kodu MAC (*Message Authentiaction Code*) zapewnia jedynie *integrity* oraz *authentiaction*.

Tworzenie podpisu cyfrowego z wykorzystaniem RSA wygląda odwrotnie niż komunikacja szyfrowana. To znaczy: podpis tworzony jest z wykorzystaniem klucza prywatnego - a więc tylko instancja generująca klucze może wiadomość podpisać. Weryfikacja odbywa się z wykorzystaniem klucza publicznego - czyli każda instancja, której nasz klucz udostępnimy, może podpis zweryfikować. 

Na początek zaimportujmy niezbędne biblioteki.

In [57]:
# !pip install pycryptodome
import Crypto.Util.number as cu
import hashlib as hl
import math
import numpy as np

**Zadanie 1**

Odpowiedz na poniższe pytania. Rozważ sytuację, w której dwie instancje komunikują się ze sobą, a trzecia próbuje dokonywać nieautoryzowanych zmian w treści wiadomości na drodze jej przesyłu.
1. Dlaczego haszowanie zapewnia tylko integrity? Podpowiedź: czy haszowanie uwzględnia jakikolwiek klucz prywatny?
2. Dlaczego kod MAC nie zapewnia *non-repudiation*? Co (niepożądanego) może dokonać odbierający wiadomość i atakujący komunikację? Podpowiedź: czy kody MAC, w szczególności popularny kod HMAC - który zakłada użycie klucza prywatnego - w jakikolwiek sposób weryfikuje KTO i DO CZEGO go użył? Kto dysponuje, a kto NIE dysponuje kluczem prywatnym HMAC?
3. Dlaczego podpis cyfrowy zapewnia wszystkie te trzy cechy?

## Generowanie kluczy

Algorytm generowania kluczy RSA może zostać przedstawiony w następujący sposób:

1) Znajdź dwie różne i kryptograficznie bezpieczne liczby pierwsze.

2) Oblicz $n = p * q$.

3) Oblicz $f = (p - 1) * (q - 1)$.

4) Znajdź dowolne $e$, takie, że $1 < e < f$ oraz $GCD(f, e) = 1$. GCD to największy wspólny dzielnik. Para $(e, n)$ to jest **klucz publiczny**.

5) Oblicz $d = e^{-1}$ mod $f$. Para $(d, n)$ to **klucz prywatny**, przy czym tajne jest tylko $d$. 

W ten sposób generowane parametry byłyby matematycznie poprawne, lecz kryptograficznie niebezpieczne. Ustalmy więc, że chcemy aby nasz klucz publiczny był odpwowiednio długi. Będzie to długość bitowa parametru $n$, oznaczmy ją jako $nlen = 2048$. Parametr $nlen$ zawsze przyjmuje parzyste wartości. Mając to założenie, musimy (**uwzględniając wszystkie założenia z algorytmu generowania kluczy**) dodatkowo zapewnić, że:

1. $65537 ≤ e < 2^{256}$
2. $LCM(p - 1, q - 1) \geq e * 2^{nlen/2}$
3. $2^{(nlen - 1)//2} < p < 2^{nlen/2}$
4. $2^{(nlen - 1)//2} < q < 2^{nlen/2}$
5. $|p - q| > 2^{(nlen/2) - 100}$

Gdzie LCM oznacza *Least Common Multiple*, czyli najmniejszą wspólną wielokrotność. Funkcję LCM znajdziesz w bibliotece math. Do potęgowania **nie używaj** pythonowej notacji "**", tylko metody pow() - przetestuj obie te metody obliczania potęgi i porównaj wydajność (zadanie opcjonalne). Do obliczania wartości bezwzględnej użyj metody abs() - również standardowa metoda pythona. Resztę niezbędnych metod znajdziesz w bibliotece [Crypto.Util.number](https://pycryptodome.readthedocs.io/en/latest/src/util/util.html) zaimportowanej jako cu. Opis powyższych założeń możesz znaleźć w [tym](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Br2.pdf) dokumencie NIST-owym.

**Zadanie 2**

Uwzględniając wszystko powyższe, napisz metodę/metody generujące wartości $n$, $e$ oraz $d$.

In [58]:
# your code here
def generate_rsa_pair(nlen):
    assert nlen % 2 == 0

    # 1, 2
    p, q = 1, 1
    n = p * q
    while not p != q or\
          not len(bin(n)) - 2 != nlen or\
          not (nlen - 1)//2 < math.log2(p) < nlen/2 or\
              not (nlen - 1)//2 < math.log2(q) < nlen/2 or\
                not math.log2(abs(p - q)) > nlen/2 - 100 :
        p = cu.getPrime(nlen // 2)
        q = cu.getPrime(nlen // 2)
        n = p * q

    # 3
    f = (p - 1) * (q - 1)
    # 4
    # e = f - 1
    e = 65537
    while math.gcd(e, f) != 1:
        e += 2
    
    d = cu.inverse(e, f)

    return (d, n), (e, n)
    

In [59]:
generate_rsa_pair(2048)

((9855319013780190897338674181995065410349132093974266114341000358392723945158896667366545954299548367141197181367718112425481543222567794625485152766966751128386063200889095810122689257037685055462839004771696011473989101764744396684322018154341970322932871205962804459601079410395367098790888556699764087966465212190560989651200730427676248605180470861322264638583006280390433168115375623337773100656964271759336333517103665845210178899397939339389237538355272196178461600992939624573192567062087755786730782323374447901673274215034794179327001512342014495795648149412112343351747388470989594581652830831601072671985,
  131326103494390706119898477057746858972398656021062884456826916450730744824097964883331568908732768357800138195944886733770237871208455442310280276704627702960643614329768751191088342450180709503450058935326469339183201936146568566853810764869486744958328571474357349410124830490643156755206877178730519099074779667768506285417855196849608090951469240693372153703317594991781753

## Naiwne szyfrowanie i deszyfrowanie


Naiwny algorytm szyfrowania wiadomości **M** z wykorzystaniem RSA:

1) Zakoduj $M$ jako liczbę.

2) Oblicz: $C = M^e$ mod $n$.

Naiwny algorytm deszyfrowania kryptogramu **C** z wykorzystaniem RSA:

1) $M = C^d$ mod $n$.

2) Zdekoduj wiadomość $M$ do jej pierwotnej postaci (np. stringa). 


**Zadanie 3**

Napisz metody szyfrujące i deszyfrujące wiadomość $M$ zgodnie z powyższym algorytmem. Zaszyfruj wiadomość, zdeszyfruj i wypisz oryginalny tekst na ekranie. Odpowiedz na pytanie: jaki warunek musi spełniać liczbowa reprezentacja wiadomości $M$, aby można ją było poprawnie zaszyfrować i zdeszyfrować?

In [60]:
def encrypt_with_rsa(M, public):
    m = cu.bytes_to_long(M)
    mlen = len(M)
    C = pow(m, *public)
    return cu.long_to_bytes(C, mlen)


def decrypt_with_rsa(C, private):
    clen = len(C)
    C = cu.bytes_to_long(C)
    M = pow(C, *private)
    return cu.long_to_bytes(M, clen)

In [61]:
priv_k, pub_k = generate_rsa_pair(2048)

In [62]:
M = 'Ala ma kota'

print(f"message: {M}")
C = encrypt_with_rsa(M.encode(), pub_k)
print(f"cipher   : {C}")
D = decrypt_with_rsa(C, priv_k)
print(f"decrypted: {D.decode()}")

print(D)
print(M.encode())

message: Ala ma kota
cipher   : b'\x00\x00\x00\x00\x00\x00\x00\x00\x1b\x81\xfdX\x99\\\x8f6MS\x17Y\xc3\x80\x80\x07\xf3[\xa5\xf4\xc7\xccc\xd5F\xb0>ni\xa5t\xa6y\xc1+\xe9P^\x98\x83\xda\xad\x1d\x13\xc5\xb4[0\xbf\xcc\x8cF\x01\x9f\xe4G?\xf5\x87\x9bp\xd7\x16\x02K\x80j\x01^\xbaL,w\x92\xc2\x9df\xaf[\x9fV\xf8\xd0>\xbf\xb5\xf7\xce\xec\xd5\x96@\xcf\xa7F\xd1\x970\xb5:\x15?A8)\xaec\x98\xd2,\xfb\xd9\xf0\xfa\xfa\x83\xe4\xf3\x93\xd3\xd0\xfaI\xa2d6\xa4\xaf\x08\x87a\xb0\xaf\xc6Q\xf2\xac5\xc0u!\xa40\xa9*\xf9\xad\xef\x99\x80\xd2\x18\xef\xc1\xf9\xa3\x86Y\x03&\xcd\xce/Y\xea$:\x04p\xa6\xe4\x1dp\xed3\x94:(r\xffLE\xec\x0f\xd4t)/\xed\xb6\x9c\xa0\xb5T\xc65\xe4\xb8\xd7\xb0\x10\xa0\xf7\xb733U\xdf\xea\xae2\\\xcdb\xc6U\x04B\xb1\x9f\xcb\n\x12 j8DoM5?\x9c\x87;D\xc1\xd9\x1c\xa9\xf2)\xe5\x1e\xa3G\xe0\x10\xe9\x0f\xe3\xcb\xf2\xf5t\xbb\x93'
decrypted:                                                                                                                                                                                 

## Naiwny schemat podpisu cyfrowego

Naiwna metoda tworzenia podpisu z wiadmości $M$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $SIG = h^d$ mod $n$.

4) Wyślij parę $(M, SIG)$ weryfikującemu.

Naiwna metoda weryfikacji podpisu $(M, SIG)$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $VER = SIG^e$ mod $n$.

4) Jeżeli $VER = h$, weryfikacja przebiegła pomyślnie, a w przeciwnym razie niepomyślnie.

**Zadanie 4**

Zaimplementuj naiwną metodę tworzenia i weryfikowania podpisu cyfrowego RSA.

In [63]:
# Your code here
def naive_fingerprint(M, priv_k):
    d, n = priv_k
    h = hl.sha3_256(M.encode()).hexdigest()
    h = int(h, 16)
    sig = pow(h, d, n)
    return M, sig
def naive_verify_fingerprint(M, sig, pub_k):
    e, n = pub_k
    h = hl.sha3_256(M.encode()).hexdigest()
    h = int(h, 16)
    ver = pow(sig, e, n)
    return ver == h

In [64]:
M, sig = naive_fingerprint(M, priv_k)
naive_verify_fingerprint(M, sig, pub_k)

True

## MGF 1

W dalszej części laboratoriów będziemy potrzebowali generować maskę. Jedynym zatwierdzonym algorytmem który do tego służy jest *Mask Generation Function 1*, opisany w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017). Jest on stosunkowo prosty. 

Parametry wejściowe:

1) M - bajty wiadomości.

2) len - pożądana długość zwórconej maski w bajtach.

3) H - wybrana funkcja skrótu, zwracająca $n$ bitowy skrót. Niech to będzie SHA-256. Dla wygody przyjmijmy też, że $hlen = n / 8$ oznacza liczbę bajtów zwracaną przez naszą funkcję skrótu.

Wyjściem funkcji są bajty tworzące maskę.

Algorytm MGF-1:

1) Dla 32-biotwego integera $i = 0, ..., ⌈ \frac{len}{hlen}⌉ - 1$ wykonuj kroki 2 i 3.

2) Oblicz tmp = H(M || i). Znak || to konkatenacja i chodzi tu o bajty wiadomości M oraz reprezentację w bajtach 32-bitowego itegera $i$.

3) Oblicz output = output || tmp.

4) Zwróc $len$ wiodących bajtów zmiennej output.

**Zadanie 5**

Zaprogramuj i przetestuj dla dowolnych wartości funkcję MGF1.

In [65]:
# Your code here
def mgf1(M, rlen=256, H=None, hlen=0):
    if H is None:
        H = lambda b : hl.sha3_256(b).digest()
        hlen = 256 / 8 # TODO
    if type(M) == str:
        M = M.encode()

    out = bytes()
    for i in range(math.ceil(rlen / hlen)):
        tmp = H(M + cu.long_to_bytes(i, 4))
        out += tmp
    return out[:rlen]

In [66]:
# mgf1 tests
for _ in range(100000):
    rl = 223
    m = cu.Random.get_random_bytes(32)
    h = mgf1(m, rlen=rl)
    if not len(h) == rl:
        print(m)
        print(len(h), rl)
    assert len(h) == rl

In [67]:
l = 1000
rl = len(mgf1(M.encode(),  rlen=l))
print(rl)
assert rl == l
mgf1(M.encode())

1000


b'\xf9c\xbdZ\x96\xed\x14\xb7\xf27\xd1\x15 Z\xc0\x08\x06)\x1b$\x13U;|$\x81\xa4\x1d\xad\x8e\xae\xd7#$1\x0e\xfe"\xa3\xe35\xc0\x11\x00S\x93\xbcr\x05\x9dE\xe4\xb7}F\x1e\x98\xf8>s\xa9\xa4V\x8d\x0f0v\x1f]T\xfaN\xb5\xa7\xe4\x01\x02\x8f\xdc|\x84\xab\n\xda\xe1\x80\x81SH\thu\x96\xa5\xd9)\x07z\xd6\xc7\xb1\xfd\x05\xb2\xdb!*$=\xaf\xb0\xb2\xc8Qk\x97H\xfb\xc7K\xfc\xdb\xdef\xc2\xfa\xc6\xcd\x8f\xe6\xea\xcf\x8d\x8a\x05SOd\xfb9:\xc5\xf9^\xc5\xaf\xd1\xe51\xfd\xecaD\x02\xc1\x18`\xa1\xdc`\x93\xb0o2\x01%-\xc6\x1c\xbe\x9ee\'\xe8\x93[\xdc\xee\xf3K\xb7\xa6<Y\x96C\x05\n\x91t.+0\xb9\xb3\x90VW\xc9\xd87\xd5\xf3;L\xa7v\x04V|\n\xfd\x19\x16r\x00n\x07\x8d\xfd}\x95\xe1\x152#\xca\xf7\x8dW\xdb\xa1\xb3\xaeEl\xc9\x9a\xc5\x9b\x9b\xb9\x0c-q\xdb\x9f|\x14V\xd7\xba\x10\xef\xb5\x16'

## OAEP

Nasz schemat ma na ten moment jedną sporą wadę, mianowicie rozmiar szyfrowanej wiadomości może być zbyt mały, czyniąc algorytm mniej bezpiecznym. Aby tego uniknąć, używamy algorytmu paddingu opisanego w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017#section-8), który zwie się *Optimal Assymetric Encryption Padding*.

### OAEP encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - długość liczby $n$ wyrażona w bajtach.

3) $mlen$ - długość wiadomości wyrażona w bajtach.

4) $M$ - bajty wiadomości.

5) $mgf1$ - Mask Generation Function 1.

Algorytm:

1) Jeżeli $mlen > k - 2*hlen - 2$ zwróc błąd.

2) Oblicz: $lHash = H("")$.

3) Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.

4) Oblicz: $DB = lHash || PS || 0x01 || M$. Długość $DB$ powinna wynosić $k - hlen - 1$ bajtów.

5) Wygeneruj losową tablicę bajtów $seed$ o rozmiarze $hlen$.

6) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

7) Oblicz: $maskedDB = DB ⊕ dbMask$.

8) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

9) Oblicz: $maskedSeed = seed ⊕ seedMask$.

10) Oblicz: $EM = 0x00 || maskedSeed || maskedDB$. Długość $EM$ powinna wynosić $k$.

11) Zwróc $EM$.

### OAEP decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - rozmiar EM wyrażony w bajtach.

3) $mgf1$ - Mask Generation Function 1.

4) $EM$ - bajty zakodowanej wiadomości.

Algorytm:

1) Rozpakuj tablicę bajtów $EM$. Jej pierwszy bajt (najbardziej znaczący) przypisz do $Y$. Kolejne $hlen$ bajtów przypisz do $maskedSeed$, resztę do $maskedDB$. Czyli $EM = Y || maskedSeed || maskedDB$.

2) Oblicz: $lHash = H("")$.

3) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

4) Oblicz: $seed = maskedSeed ⊕ seedMask$.

5) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

6) Oblicz: $DB = maskedDB ⊕ dbMask$.

7) Rozpkauj tablicę bakjtów $DB$. Pierwsze (najbardziej znaczące) $hlen$ bajtów przypisz do zmiennej $lHash'$. Następne $k - mlen - 2*hlen - 2$ bajtów do PS. Kolejny pojedynczy bajt powinien wynosić 0x01, jeżeli jest inaczej zwróć błąd i **zakończ działanie**. Resztę bajtów przypsiz do zmiennej $M$. Czyli: $DB = lHash' || PS || 0x01 || M$. 

8) Jeżeli $Y \neq 0x00$ zwróć błąd i **zakończ działanie**.

9) Jeżeli $lHash \neq lHash'$ zwróć błąd i **zakończ działanie**.

10) Zwróc $M$.

**Zadanie 6**

Zaproogramuj kodowanie i dekodowanie OAEP. Zmodyfikuj algorytm szyfrowania RSA, tak, aby przed zaszyfrowaniem wiadomość była paddingowana. Zmodyfikuj algorytm deszyfrowania tak, aby po zdeszyfrowaniu konieczne było wywołanie metody dekodowania OAEP w celu odzyskania wiadomości.

In [68]:
M = "Ala ma kota"

# Your code here
def enc_with_oaep(M, pub_k, k=256, H=None, hlen=None):
    if H is None:
        H = lambda mb: hl.sha3_256(mb).digest()
        hlen = 256 // 8

    if type(M) == str:
        Mb = M.encode()
    else:
        Mb = M
        
    mlen = len(Mb)
    assert not mlen > k - 2 * hlen - 2

    lHash = H("".encode())
    PS = bytes([0x00] * (k - mlen - 2*hlen - 2))
    DB = lHash + PS + bytes([0x01]) + Mb
    assert len(DB) == k - hlen - 1

    seed = [cu.getRandomInteger(8) for _ in range(hlen)]
    seed = bytes(seed)
    dbMask = mgf1(seed, k - hlen - 1)
    maskedDB = bytes(np.uint8(list(DB)) ^ np.uint8(list(dbMask)))
    seedMask = mgf1(maskedDB, hlen)
    maskedSeed = bytes(np.uint8(list(seed)) ^ np.uint8(list(seedMask)))
    EM = bytes([0x00]) + maskedSeed + maskedDB
    assert len(EM) == k

    return encrypt_with_rsa(EM, pub_k)
    # return cu.long_to_bytes(pow(cu.bytes_to_long(EM), *pub_k))


def dec_with_oaep(EM: bytes, priv_k, mlen, k=256, H=None, hlen=None):
    if H is None:
        H = lambda mb: hl.sha3_256(mb).digest()
        hlen = 256 // 8
    
    EM = decrypt_with_rsa(EM, priv_k)

    Y = EM[0]
    maskedSeed = EM[1:hlen+1]
    maskedDB = EM[hlen + 1:]
    lHash = H("".encode())
    seedMask = mgf1(maskedDB, hlen)
    seed = bytes([a ^ b for a, b in zip(maskedSeed, seedMask)])
    dbMask = mgf1(seed, k - hlen - 1)
    DB = bytes([a ^ b for a,b in zip(maskedDB, dbMask)])

    lHash_ = DB[:hlen]
    offset = hlen
    nex = k - mlen - 2*hlen - 2
    PS = DB[offset: offset + nex]
    offset += nex
    assert DB[offset] == 0x01
    offset += 1
    M = DB[offset:]

    assert Y == 0x00
    assert lHash == lHash_

    return M


In [78]:
priv_k, pub_k = generate_rsa_pair(2048)

EM = enc_with_oaep(M, pub_k)
D = dec_with_oaep(EM, priv_k, mlen=len(M))
print(f"Message: {M}")
print(f"Aft. OAEP: {EM}")
print(f"Ret. OAEP: {D}")

Message: Ala ma podpis
Aft. OAEP: b"Z\x9b6 \x01Xa\xb6W\xe78\x99\xb8>T\x01\x01\xe8\x81\x9a-\xab\x93\x8c\xb7IG)[\x11,q\x14]f\x0cf^\x00\xab^a\xe8\x11\xe6\x9cx&\x904\xf4\x91zig\x9bn\xee\x0c8-\x98\x19 \xe0\x8f\x1d\xb4:\x06\xb3\xff\xfdM\xad8\xdc\xbe\xeb}&\x1ds\xe5\xb4\xc6\xc5`|\xb5pN}\xec\xefnhH\r\x8f|\xc6\xd73)b>\r\xda,\xbd[\xc9\x12\xc0\xb0\xfc\xfd\xcf\xdb\xe2\xae\x08\xad\xeb1\\[\x14\x17\xa1\x10(\xe9\x15\xce\x1co\xd6\xbao$GeiWi\r1\xcbp\x04U\xb7\xfeJ\xa7`\x07q\xd7\xce\xacK/Sa\xe3\xb3\x94\xab\x8eI\x10y\xedZ\xa3\xbbK:\n]\xdfI\xca`>`:kt7\x85@\x88=\xaaE7\x8eL?\xcdV\xcfZJ\xfa\n*\xb4\\\x8e\xea-\xaa\x82\x80+\x96P\xf4\xf6\xb9<%q\xaf1\x00'\xcb\x87\x1c\x15\xb3\xf9\xd7\x93b\x1d\xbd\xf5\xa4\xff8F\xe1\x99\x04F^\xccoE"
Ret. OAEP: b'Ala ma podpis'


## EMSA - PSS

Utworzenie bezpiecznej sygnatury RSA wymaga zastowania algorytmu *Encoding Method for Signature with Appendix - Probabilistic Signature Scheme* .

### EMSA encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $M$ - bajty wiadomości do podpisania.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - pożądana długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.


Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd**.

3) Wygeneruj tablicę losowych bajtów $salt$ o długości $slen$.

4) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

5) Oblicz: $mHash' = H(M')$.

6) Wygeneruj tablicę $PS$ składającą się z bajtów 0x00 o długości $emlen - slen - hlen - 2$.

7) Oblicz: $DB = PS || 0x01 || salt$. Długość $DB$ powinna wynosić $emlen - hlen - 1$ bajtów.

8) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

9) Oblicz: $maskedDB = DB ⊕ dbMask$.

10) Ustaw $8 * emlen - emBits$ **najbardziej znaczących** bitów $maskedDB$ na wartości 0.

11) Oblicz: $EM = maskedDB || mHash' || 0xbc$.

12) Zwróć $EM$.


## EMSA decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $EM$ - sygnatura wiadomości $M$.

4) $M$ - bajty wiadomości do weryfikacji.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.

Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd weryfikacji**.

3) Jeżeli ostatni bajt (najmniej znaczący) $EM$ nie ma wartości 0xbc **zakończ i zwróć błąd weryfikacji**.

4) Podstaw $emlen - hlen - 1$ najbardziej znaczących bajtów do $maskedDB$ oraz kolejne $hlen$ bajtów do $mHash'$.

5) Jeżeli $8 * emlen - emBits$ najbardziej znaczących bitów $maskedDB$ nie ma wartości 0, **zakończ i zwróć błąd weryfikacji**.

6) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

7) Oblicz: $DB = maskedDB ⊕ dbMask$.

8) Ustaw $8 * emlen - emBits$ najbardziej znaczących bitów $DB$ na 0.

9) Jeżeli $emlen - hlen - slen - 2$ najbardziej znaczących bajtów $DB$ nie posiada wartości 0x00 lub gdy bajt na pozycji $emlen - hlen - slen - 1$ (licząc od najbardziej znaczącego) nie posiada wartości 0x01 **zakończ i zwróć błąd weryfikacji**.

10) Przypisz do zmiennej $salt$ dokładnie $slen$ najmniej znaczących bajtów $DB$.

11) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

12) Oblicz $mHash'' = H(M')$.

13) Jeżeli $mHash' \neq mHash''$ **zakończ i zwróć błąd weryfikacji**, w przeciwnym razie **weryfikacja powiodła się**.


**Zadanie 7**

Zaprogramuj kodowanie i dekodowanie EMSA, a następnie popraw algorytmy tworzenia i weryfikacji podpisu cyfrowego RSA. Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest popdis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [70]:
# Your code here
def enc_emsa_pss(M, slen, emBits, emlen, H=None):
    if H is None:
        H = lambda mb: hl.sha3_256(mb).digest()
        hlen = 256 // 8
    
    if type(M) == str:
        M = M.encode()
        
    mHash = H(M)
    assert emlen >= hlen + slen + 2
    salt = cu.Random.get_random_bytes(slen)

    M_ = bytes([0x00] * 8) + mHash + salt
    assert len(M_) == 8 + hlen + slen
    mHash_ = H(M_)
    PS = bytes([0x00] * (emlen - slen - hlen - 2))
    DB = PS + bytes([0x01]) + salt
    assert len(DB) == emlen - hlen - 1
    assert DB[emlen - hlen - slen - 2] == 1
    
    dbMask = mgf1(mHash_, rlen=(emlen - hlen - 1))
    # print(len(mHash_), len(dbMask), emlen - hlen - 1)
    assert len(dbMask) == emlen - hlen - 1
    maskedDB = np.array([a ^ b for a,b in zip(DB, dbMask)], dtype=np.uint8)
    
    ft = (8*emlen - emBits) // 8
    rt = (8*emlen - emBits) % 8
    maskedDB[:ft] = 0
    maskedDB[ft] <<= rt
    maskedDB[ft] >>= rt

    maskedDB = bytes(maskedDB)
    EM = maskedDB + mHash_ + bytes([0xbc])
    assert EM[-1] == 0xbc
    return EM

def dec_emsa_pss(EM, M, slen, emBits, emlen, H=None):
    if H is None:
        H = lambda mb: hl.sha3_256(mb).digest()
        hlen = 256 // 8
    
    if type(M) == str:
        Mb = M.encode()
    else:
        Mb = M
        
    mHash = H(Mb)
    assert emlen >= hlen + slen + 2
    assert EM[-1] == 0xbc
    s = emlen - hlen - 1
    maskedDB, mHash_ = EM[:s], EM[s:s+hlen]

    ft = (8*emlen - emBits) // 8
    rt = (8*emlen - emBits) % 8

    assert sum(list(maskedDB[:ft])) == 0
    assert maskedDB[ft] >> (8 - rt) == 0
    dbMask = mgf1(mHash_, emlen - hlen - 1)
    DB = np.array([a ^ b for a,b in zip(maskedDB, dbMask)], dtype=np.uint8)

    DB[:ft] = 0
    DB[ft] <<= rt
    DB[ft] >>= rt

    if DB[:emlen - hlen - slen - 2].sum() > 0:
        print(emlen - hlen - slen - 2)
        print(list(DB[:emlen - hlen - slen - 2]))
    assert DB[:emlen - hlen - slen - 2].sum() == 0
    assert DB[emlen - hlen - slen - 2] == 1
    
    DB = bytes(DB)
    salt = DB[-slen:]
    M_ = bytes([0x00] * 8) + mHash + salt

    assert len(M_) == 8 + hlen + slen
    mHash_2 = H(M_)
    return mHash_ == mHash_2

In [71]:
# emsa debug
M = "Ala ma podpis"
for i in range(1000):
    # print(f"iter-{i}")
    EM = enc_emsa_pss(M, slen=256//8, emBits=2047, emlen=2048//8)
    # print(EM)
    r = dec_emsa_pss(EM, M, slen=256//8, emBits=2047, emlen=2048//8)
    assert r

In [72]:
# debug RSA
# M = "Ala ma podpis"
# priv_k, pub_k = generate_rsa_pair(4096)

# for _ in  range(1000):
#     EM = enc_emsa_pss(M, slen=256//8, emBits=2047, emlen=2048//8)
#     assert dec_emsa_pss(EM, M, slen=256//8, emBits=2047, emlen=2048//8)
    
#     em_long = cu.bytes_to_long(EM)
#     assert cu.long_to_bytes(em_long, len(EM)) == EM
    
#     enc_em_long = pow(em_long, *pub_k)
#     dec_em_long = pow(enc_em_long, *priv_k)

#     decEM = cu.long_to_bytes(dec_em_long, len(EM))

#     eq = (decEM == EM)
#     if not eq:
#         print(type(EM), EM)
#         print(type(decEM), decEM)
#         print(enc_em_long)
#         print(dec_em_long)
#         print(em_long)
#     assert em_long == dec_em_long
#     assert eq

#     assert len(decEM) == 2048 // 8
#     assert dec_emsa_pss(decEM, M, slen=256//8, emBits=2047, emlen=2048//8)

In [73]:
def rsa_fingerprint(M, priv_k):
    d, n = priv_k
    EM = enc_emsa_pss(M, slen=256//8, emBits=2047, emlen=2048//8)
    EM = cu.bytes_to_long(EM)
    eEM = pow(EM, d, n)
    return eEM

def verify_rsa_fingerprint(M, eEM, pub_k):
    e, n = pub_k
    EM = pow(eEM, e, n)
    EM = cu.long_to_bytes(EM)
    assert len(EM) == 2048 // 8
    return dec_emsa_pss(EM, M, slen=256//8, emBits=2047, emlen=2048//8)

In [74]:
priv_k, pub_k = generate_rsa_pair(4096)

In [75]:
eEM = rsa_fingerprint(M, priv_k)
print(EM)
verify_rsa_fingerprint(M, eEM, pub_k)

b'f\x1a\xd8m\rCy=\xe5Z\x8d@\x13\x9f\x1bn\xef8\xff\x13d\xde2\x00\xca6{\xd3\xcc\x1c\xaaDr\xb7\x9a\x18\xd9\xc1s\xfcG\xb8\xd0ne0\xfa\xf4\xd4e\xc7\x907\x93ZF\x9c9\xb8\xa0\xcb\xc1\x0e\xe1`\x8d\xf9\xab\xd1\xcd\x0f\x9b\x91Gw.,Tp\x1a%\x9b\xbc\xc4\xc6\xfb6RR_\x0b\x84\x97\xf89\t<dy\x8dMJ\x0f\x9e\xa6\xee\xeb\x07\x0e&\xc4\xcd\xbdh~\xcb\xfc\xa5\x10o\x97\xeb\xa8-\xbad\x91\x01\x15k\xee\x9d\xa7\xa3\xfcMc6\xb9\xb2R.\xa0\xf0M4e9\x83:\x07\x8b\xb8K\xf2\x17U\\^\xf0\xd8%\xc2\xc0\xd8\x90\xb5\xac3\xc3,\x10\x8dC\xa3\xdb\x9b,4AU)52\x0b\x81\x9ch\xbd\x19U\x7f\xfd e\x98\xcf\x93\xfb{0\x85m\xcd\xd3u\x1c\x1a\x8c\xf7\xfbt\x11\\\xc2\xde^\xc0\x08\x81\x14Q\x8b\xe92\x9bg\xf3\xd2D\xfd\xc2\xe3\x83\x86\xf5TX\x83\xe0\x00@^\xa9\xcd$\x91\xc5\x98\x89\x8a\xc2\x9c\t\xc9\xbc'


True

## RSA - KEM

RSA - KEM, czyli z ang. *Key Encapsulation Mechanism* to prosty algorytm dedykowany do wymiany klucza symetrycznego. Obie strony dysponują uzgodnioną funkcją skótu H. Instancja, która chce **otrzymać** tajny klucz do komunikacji symetrycznej generuje klucze RSA i udostępnia swój klucz publiczny. Instancja, która chce wygenerować tajny klucz do komunikacji symetrycznej dysponuje kluczem publicznym instancji, która chce go otrzymać.

Instancja generująca klucz symetryczny:

1) Znajdź losową liczbę $RAND$ spełniającą warunki OAEP.

2) Oblicz: $KEY = H(RAND)$. Jeżeli trzeba, przytnij $KEY$ do odpowiedniej długości.

3) Oblicz: $CIPHERED\_KEY = RSA\_OAEP\_ENCODING(KEY, (e, n))$.

4) Wyślij $CIPHERED\_KEY$.

Instancja otrzymująca zaszyfrowany klucz symetryczny:

1) Oblicz: $KEY = RSA\_OAEP\_DECODING(CIPHERED\_KEY, (d, n))$

2) Jeżeli trzeba przytnij $KEY$ do odpowiedniej długości.

Np. AES występuje w wersji 128b, 192b i 256b. Jeżeli jako H przyjmiemy więc SHA-256, nie trzeba przycinać klucza dla algorytmu AES-256. W przeciwnym razie należy klucz odpowiednio przyciąć (z lewej lub prawej, byle obie strony tak samo) i to ta wartość staje się kluczem symetrycznym.

**Zadanie 8**

Zasymuluj takową wymianę (bez przycinania klucza).

In [76]:
# Your code here

H = lambda mb: hl.sha3_256(mb).digest()
def gen_ins(pub_k):
    rand = cu.Random.get_random_bytes(32)
    key = H(rand)
    return key, len(key), enc_with_oaep(key, pub_k)

def res_ins(secret, priv_k, mlen):
    key = dec_with_oaep(secret, priv_k, mlen=mlen)
    return key

In [77]:
priv_k, pub_k = generate_rsa_pair(2048)
key, mlen, secret = gen_ins(pub_k)
key2 = res_ins(secret, priv_k, mlen)

assert key == key2